In [1]:
import os
import numpy as np
import multiprocessing as mp
from mg5_tools import semilla, run_mg5_file
from mg5_tools import get_seeds_from_mg5_output_folder as get_used_seeds

UFO_name = os.path.join(os.path.dirname(os.getcwd()), '00_FeynRules_UFO','Mod2_VLQ_UFO')
cases = ["woRHC"]

mass_step = 500 # GeV
g_u_step = 1/4

# Leptoquark parameters
M_U = np.arange(1000, 3500 + mass_step, mass_step)
g_U = np.arange(0.5, 3.5 + g_u_step , g_u_step)

## Leptoquark parameters (fixed)
#M_U = np.array([1000, 1500, 2000, 2500, 3000, 3500])
g_U = np.array([1.75])

# Z' parameters
mass_constraints_zp = {"zp_upper_limit": M_U*np.sqrt(3./2.), "zp_lower_limit": M_U*np.sqrt(1./2.)}
g_zp_step = 1/2
g_zp = np.arange(0.5, 3.5 + g_zp_step, g_zp_step)

output_folder= os.path.join(os.sep, "disco2", "SIMULATIONS", f"lq_zp")
os.makedirs(output_folder, exist_ok=True)

n_events = 1.0e6

nevts = 25e3
n_runs = int(n_events/nevts)

#n_runs = 1 

n_cores = mp.cpu_count()

n_workers = 3 

n_cores = int(n_cores/n_workers)

if n_cores == 1:
    raise ValueError("n_cores must be greater than 1")

UFO_name

In [2]:
headers = {
"total_tau_tau" : 
f"""
import model {UFO_name} 
define lq = vlq vlq~
define ta = ta+ ta-
generate p p > ta ta @0 QED = 0
"""
}

kin_gen_cuts = {
    "cut_decays" : True,
    "ptb" : 30,
    "ptj" : 20,
    "ptl" : 20,
    "etab" : 2.5,
    "pt_min_pdg" : "{15: 30}",
    "eta_max_pdg" : '{15: 2.5}',
    'mxx_min_pdg' : '{15:100}'
}

In [3]:
def prepare_proc_script(x, output_folder):
    case = x[0] 
    header = x[1]
    mass_constraint_zp = x[2][0]
    gz = x[3]
    gu = x[4]
    m = x[5]
    label = f"mu_{m}_gu{gu}_gzp_{gz}"
    mg5_output_folder = os.path.join(output_folder, case, mass_constraint_zp, header, label.replace(".", "_"))
    os.makedirs(mg5_output_folder, exist_ok=True)

    proc_file_path = os.path.join(mg5_output_folder, f"proc_script.mg5")
    with open(proc_file_path, 'w') as f:
        f.write(headers[header])
        f.write(f"output {mg5_output_folder} -nojpeg\n")

    # check if the process has already been generated
    if not os.path.exists(proc_file_path):
        raise Exception(f"Process {proc_file_path} not generated")
    return run_mg5_file(proc_file_path)

In [4]:
from itertools import product
from multiprocessing import Pool
from tqdm.notebook import tqdm as tq
prod = list(product(cases, headers.keys(), mass_constraints_zp.items(), g_zp, g_U, M_U,[output_folder]))

pb = tq(total=len(prod))
ncores = 13
n_batches = int(len(prod)/ncores)
for i in range(n_batches+1):
    batch = prod[i*ncores:(i+1)*ncores]
    with Pool(ncores) as p:
        all(list( p.map ( prepare_proc_script, batch )))
    pb.update(len(batch))
pb.close()

True

In [5]:
import os
from pathlib import Path
from random import random
from time import sleep

def generate_events(x):
    #add random sleep to fix printing issues in multiprocessing pool
    sleep(2+random()*3)
    case = x[0] 
    header = x[1]
    mass_constraint_zp = x[2][0]
    gz = x[3]
    gu = x[4]
    m = x[5]
    output_folder  = x[6]    
    label = f"mu_{m}_gu{gu}_gzp_{gz}"
    param_card = os.path.join("Param_Cards", case, mass_constraint_zp, f"param_card_{label}.dat")
    delphes_card = os.path.join(os.getcwd(), "delphes_card_CMS-0.7bTag.dat")
    pythia_card = os.path.join(os.getcwd(), "pythia8_card.dat")

    # check if cards exist
    if not os.path.exists(param_card):
        raise Exception(f"Param card {param_card} not generated")
    elif not os.path.exists(delphes_card):
        raise Exception(f"Delphes card {delphes_card} not exists")
    elif not os.path.exists(pythia_card):
        raise Exception(f"Pythia card {pythia_card} not exists")
    
    mg5_output_folder = os.path.join(output_folder, case, mass_constraint_zp , header, label.replace(".", "_"))
    # check if the process has already been generated
    if not os.path.exists(mg5_output_folder):
        raise Exception(f"Process {mg5_output_folder} not generated")
    seeds = get_used_seeds(mg5_output_folder)
    seed = semilla(seeds)

    f = open(os.path.join(mg5_output_folder, f"run_{seed}.mg5"), 'w')
    f.write(f"launch {mg5_output_folder} -m\n")
    f.write(f"{n_cores}\n")
    f.write("shower = Pythia8 \n")
    f.write("detector = Delphes \n")
    f.write("done \n")
    f.write(f"set nevents {nevts}\n")
    f.write(f"set iseed {seed}\n")
    f.write('set sde_strategy 1\n')
    [f.write(f"set {key} {value}\n") for key, value in kin_gen_cuts.items()]
    f.write(f"{param_card}\n")
    f.write(f"{delphes_card}\n")
    f.write(f"{pythia_card}\n")
    f.write("done \n")
    f.close()
    return run_mg5_file(os.path.join(mg5_output_folder, f"run_{seed}.mg5"))

In [ ]:
import multiprocessing as mp
import tqdm.notebook as tqdm
log_file = open (os.path.join(os.getcwd(), "generation.log"), 'w')
prod = list(
    product(
        cases, 
        headers.keys(), 
        mass_constraints_zp.items(), 
        g_zp, g_U, M_U,
        [output_folder]
    )
)

pbar = tqdm.tqdm(total=n_runs*len(prod))
for run in range(n_runs):
    log_file.write(f"Run {run} started\n")
    with mp.Pool(n_workers) as pool:
        list( pool.map ( generate_events, prod ) )
    log_file.write(f"Run {run} finished\n")
    pbar.update(len(prod))
pbar.close()

log_file.write("All runs finished\n")
log_file.close()